# LABORATORIO 2: Optimización de Redes

Jairo Garavito Correa - 202111499

In [3]:
from pyomo.environ import *

from pyomo.opt import SolverFactory

import matplotlib.pyplot as plt

import pandas as pd

## Problem 1: transport networks

Sets:

* Origin cities: S = {Bogotá, Medellín}
* destination cities: D = {Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}

Parameters:

* Offer of the origin cities:
$$O_s = \forall s \in S$$

* Demand of the destination cities:
$$P_d = \forall d \in D$$

* Cost to send products from a supplier city to a customer city:
$$E_{s,d} = \forall {s \in S, d \in D}$$

Decision variables:

* Tons of products to be shipped from each city of origin to each destination city:
$$X_{s,d} = \forall {s \in S, d \in D}$$

Objective function:

* Minimize transport cost:
$$\text{Minimize} \sum_{s \in S} \sum_{d \in D} X_{s,d} \times E_{s,d}$$

Constraints:

* Demand of destination cities:

$$\sum_{s \in S}  X_{s,d} = P_d \space \forall d \in D $$

* Offer of the origin cities:

$$\sum_{d \in D}  X_{s,d} \leq O_s \space \forall s \in S $$


In [4]:
model = ConcreteModel()

# Conjuntos
model.S = ['Bogotá', 'Medellín']  # Ciudades de origen
model.D = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chía', 'Manizales']  # Ciudades de destino

# Parámetros
# Oferta en las ciudades de origen
model.O = {'Bogotá': 550, 'Medellín': 700}

# Demanda en las ciudades de destino
model.P = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chía': 225, 'Manizales': 200}

# Costos de transporte de las ciudades de origen a las de destino
model.E = {
    ('Bogotá', 'Cali'): 100000000,
    ('Bogotá', 'Barranquilla'): 2.5,
    ('Bogotá', 'Pasto'): 1.6,
    ('Bogotá', 'Tunja'): 1.4,
    ('Bogotá', 'Chía'): 0.8,
    ('Bogotá', 'Manizales'): 1.4,
    ('Medellín', 'Cali'): 2.5,
    ('Medellín', 'Barranquilla'): 100000000,
    ('Medellín', 'Pasto'): 2.0,
    ('Medellín', 'Tunja'): 1.0,
    ('Medellín', 'Chía'): 1.0,
    ('Medellín', 'Manizales'): 0.8
}

# Variables de decisión
model.X = Var(model.S, model.D, domain=NonNegativeReals)

# Función objetivo: minimizar el costo de transporte
def objective_rule(model):
    return summation(model.E, model.X)
model.OBJ = Objective(rule=objective_rule, sense=1)  # sense=1 para minimizar

# Restricciones

# Restricción de demanda en las ciudades de destino
def demand_rule(model, d):
    return sum(model.X[s, d] for s in model.S) >= model.P[d]
model.DemandConstraint = Constraint(model.D, rule=demand_rule)

# Restricción de oferta en las ciudades de origen
def offer_rule(model, s):
    return sum(model.X[s, d] for d in model.D) <= model.O[s]
model.OfferConstraint = Constraint(model.S, rule=offer_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

model.display()

Model unknown

  Variables:
    X : Size=12, Index={Bogotá, Medellín}*{Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogotá', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Chía') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogotá', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellín', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellín', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell

## Problem 2: Optimal Routes for teams

Sets:

* s

Parameters:

$$s$$

Decision variables:

$$s$$

Objective function:

$$s$$

Constraints:

$$s$$

In [17]:

#Modelo
model = ConcreteModel()

#Conjuntos
localidades = {0,1,2,3,4,5}

#Parametros
n = len(localidades)

df = pd.read_csv('./data/proof_case.csv')
df.columns = [int(col.strip(" ")) for col in df.columns]
df = df.apply(pd.to_numeric)
for i in range(len(df)):
    df.iloc[i, i] = 999

costos = {}
for origen in localidades:
    for destino in localidades:
        costos[(origen, destino)] = int(df[origen][destino])

print(costos)
         
#Variable de decision
model.x = Var(localidades, localidades, domain=Binary)

#Funcion Objetivo
model.obj = Objective(
    expr = sum(sum(model.x[(i,j)]*costos[i,j] for j in localidades) for i in localidades), 
    sense = 1
)

#Restricciones
def destiny_once(model, i):
    return sum(model.x[(i,j)] for j in localidades if j!=i) == 1
model.destiny_once = Constraint(localidades, rule=destiny_once)

def origin_once(model, j):
    return sum(model.x[(i,j)] for i in localidades if i!=j) == 1
model.origin_once = Constraint(localidades, rule=origin_once)

#Modelo
SolverFactory('glpk').solve(model)
model.display()

{(0, 0): 999, (0, 1): 1, (0, 2): 1, (0, 3): 2, (0, 4): 2, (0, 5): 1, (1, 0): 1, (1, 1): 999, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 2, (2, 0): 1, (2, 1): 3, (2, 2): 999, (2, 3): 3, (2, 4): 3, (2, 5): 1, (3, 0): 2, (3, 1): 2, (3, 2): 3, (3, 3): 999, (3, 4): 1, (3, 5): 3, (4, 0): 2, (4, 1): 3, (4, 2): 2, (4, 3): 1, (4, 4): 999, (4, 5): 3, (5, 0): 1, (5, 1): 3, (5, 2): 3, (5, 3): 2, (5, 4): 3, (5, 5): 999}
Model unknown

  Variables:
    x : Size=36, Index={0, 1, 2, 3, 4, 5}*{0, 1, 2, 3, 4, 5}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1) :     0 :   1.0 :     1 : False : False : Binary
        (0, 2) :     0 :   0.0 :     1 : False : False : Binary
        (0, 3) :     0 :   0.0 :     1 : False : False : Binary
        (0, 4) :     0 :   0.0 :     1 : False : False : Binary
        (0, 5) :     0 :   0.0 :     1 : False : False : Binary
        (1, 0) :     0 :   1.0 :     1 : False : False

In [8]:
# Lectura del archivo CSV, especificando que la primera fila es el índice
df = pd.read_csv('./data/proof_case.csv', index_col=0)

# Asegurar que los nombres de columnas también se tratan como enteros
df.columns = df.columns.astype(int)

# Revisar los datos
localidades = df.index.tolist()  # Lista de localidades

# Crear el diccionario de costos (evitando el ciclo i=i)
costos = {}
for origen in localidades:
    for destino in localidades:
        if origen != destino:
            costos[(origen, destino)] = df.loc[origen, destino]
        else:
            costos[(origen, destino)] = 999  # Evitar ciclos (ir a sí mismo)

# Verificar la correcta creación del diccionario de costos
print(costos)

KeyError: 0

## Problem 3: Optimization of Sensor Placement in Smart Cities

Sets:

* Locations: L = {L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}

* Sensors: S = {Enviromental Sensor, Traffic Sensor, Public Safety Sensor}

Parameters:

* Energy consumption cost of sensor s.
$$E_{s} \space \forall s \in S$$

* Cost of installation at the location l.
$$I_{l} \space \forall l \in L$$

* Cost of communication of a sensor s in a location l.
$$C_{s,l} \space \forall s \in S, l \in L$$

* Binary parameter indicating whether sensor s can cover location l directly.
$$k_{s,l} \space \forall s \in S, l \in L$$

* Binary parameter indicating whether location l1 is adjacent to l2.
$$A_{l_1,l_2} \space \forall l_{1} \in L, l_{2} \in L$$

Decision variables:
* Binary decision variable indicating whether sensor s is conected at location l.
$$X_{s,l} \in \{0,1\}$$

* Binary decision variable indicating whether sensor s is selected.
$$Y_{s} \in \{0,1\}$$

Objective function:

$$\text{Minimize} \sum_{s \in S} \left( E_s \times Y_s \right) + \sum_{s \in S} \sum_{l \in L} \left( C_{s,l} \times X_{s,l} \right) + \sum_{s \in S} \sum_{l \in L} \left( I_{l} \times X_{s,l} \right)$$

Constraints:

$$ \sum_{l_1 \in L} A_{l_1, l_2} \times X_{s,l_2} \geq k_{s,l_2} \space \forall l_2 \in L, s \in S$$